In [ ]:
import glob
import numpy as np
from PIL import Image
import os
from transformers import pipeline
import pandas as pd
import requests
import tqdm
import torch
from torchvision.transforms.functional import to_tensor

In [ ]:
df = pd.read_parquet("dataset_paths.parquet")
paths = list(df["LOCAL_PATH"])
df["LABEL"] = len(df)*["blah"]
df["SCORES_SUM"] = len(df)*[0.0]

In [ ]:
# Food 1 True
# Not Food 0 False
i = 0
number_of_skipped = 0
classifier = pipeline("image-classification", model="nateraw/food")
for path in tqdm.tqdm(paths):
    
    try:
        image = Image.open(path).convert("RGB")
        output = classifier(image)
    except:
        i += 1
        number_of_skipped += 1
        continue
    
    score_sum = sum([o["score"] for o in output])
    df["LABEL"][i] = str(not (score_sum <= 0.3))
    df["SCORES_SUM"][i] = score_sum
    
    i += 1
    
    if i % 1000 == 0:
        df.to_parquet("dataset_paths_annotated.parquet")
    # break

In [ ]:
df.to_parquet("dataset_paths_annotated.parquet")

In [ ]:
df